In [1]:
import urllib, json
import pandas as pd

# Json Download

In [16]:
def read_json(url):
    
    response = urllib.request.urlopen(url) 
    data = json.loads(response.read())
    
    json_next = data['next']
    
    protein_id = pd.Series([m['metadata']['accession'] for m in data['results']])
    fragments = [m['entries'][0]['entry_protein_locations'][0] for m in data['results']]
    positions = [frag['fragments'][0] for frag in fragments]
    domain_start = pd.Series([pos['start'] for pos in positions])
    domanin_end = pd.Series([pos['end'] for pos in positions])

    protein_df = pd.DataFrame({'protein_id':protein_id,'domain_start':domain_start,'domanin_end':domanin_end})
    return protein_df, json_next

In [17]:
def create_json(url):
  protein_df_global = pd.DataFrame(columns=['protein_id','domain_start','domanin_end'])
  
  read = True

  while read == True:
      protein_block, next_url = read_json(url)

      protein_df_global = protein_df_global.append(protein_block, ignore_index=True)
      if next_url != None:
          url = next_url
          read = True
      else:
          read = False

  return protein_df_global

In [18]:
reviewed_protein_df = create_json("https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam/pf00362?format=json")
unreviewed_protein_df = create_json("https://www.ebi.ac.uk/interpro/api/protein/unreviewed/entry/pfam/pf00362?format=json")

In [19]:
unreviewed_protein_df

,protein_id,domain_start,domanin_end
0,A0A016VF04,166,410
1,A0A023AZ45,42,113
2,A0A023B4T2,17,243
3,A0A023B824,93,231
4,A0A023B8X4,132,371
...,...,...,...
6703,X1U792,67,133
6704,X2BBV3,97,340
6705,X2BBX2,39,283
6706,X2JAU0,180,423
